<a href="https://colab.research.google.com/github/Emeralddddd/learning_material/blob/master/AI3_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow_datasets as tfds
import tensorflow as tf

     |████████████████████████████████| 332.1MB 56kB/s 
     |████████████████████████████████| 3.0MB 28.8MB/s 
     |████████████████████████████████| 419kB 40.0MB/s 
     |████████████████████████████████| 61kB 22.7MB/s 


In [0]:
import numpy as np
import pandas as pd
import re

#数据预处理
###读取文件
经测试，原文件可以以txt的格式打开；由于原生编码为gbk，这里转换为utf-8读取；因为文本共1.5G，可能无法在小内存的设备上一次性处理；这种情况可以考虑使用tensorflowd的dataset等方式分步地读取数据


In [0]:
with open('news_sohusite_xml.smarty.txt',encoding='utf-8') as f:
  str = f.read()

###提取信息
由于原文本为标签文本，考虑到在文本分类中标题的内容与正文有很多重复，故这里只提取了正文部分做为特征，而使用url来作为标签；使用正则表达式可以很容易地提取到这两部分信息；分别提取后储存在dataframe中以备下一步的处理


In [0]:
def load_data(filename):
  with open(filename,encoding='utf-8') as f:
    str = f.read()
    
  contentp = re.compile(r'<content>(.*?)</content>')
  content = contentp.findall(str)
  urlp = re.compile(r'<url>http://(.*?)/')
  url = urlp.findall(str)
  return pd.DataFrame({'content':content,'url':url})

In [0]:
dataset = load_data('news_sohusite_xml.smarty.txt')

###文本清洗
考虑到英文、数字、标点等可能会干扰文章内容，而本次实验计划采用基于字的神经网络，因此这里使用正则表达式去除了除汉字以外的所有文本内容，并且对字和标题分别进行了编码，对文本长度进行了填充对齐。返回了编码后的数据、标签和文本-编码转换的字典


In [0]:
def preprocess(dataset):
  cop = re.compile("[^\u4e00-\u9fa5]")
  
  allcontent = cop.sub("",str)
  tmp = list(set([i for i in allcontent]))
  dic = {tmp[i]:i for i in range(len(tmp))}
  
  dataset = dataset[dataset['content']!='']
  dataset['content'] = dataset['content'].apply(lambda x:cop.sub("",x))
  values = []
  for i in [j for j in dataset.values[:,0]]:
    values.append([dic[j] for j in i])
  values = np.array(list(map(lambda l:(l + [0]*(1000 - len(l)))[:1000], values)))
  dicl = {i:index for index,i in enumerate(set(dataset['url'].values))}
  labels = dataset['url'].apply(lambda x:dicl[x]).values
  return values,labels,dic,dicl

In [0]:
values,labels,dic,dicl = preprocess(dataset)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


#
###模型
本次实验采用了单层的双向LSTM来对文本进行处理；
实现方式采用了keras的顺序结构模型，分别使用了一个嵌入层、一个LSTM层、一个全连接层和一个输出层

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(2469, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

W0605 09:33:41.401958 140515976701824 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fcbfe61d0b8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0605 09:33:41.411154 140515976701824 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fcbfe61d748>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


###误差函数
采用了交叉熵作为误差函数
###优化算法
采用了adam优化算法

In [0]:
  loss = tf.keras.losses.sparse_categorical_crossentropy
model.compile(loss=loss,
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
data.output_shapes

In [0]:
model.fit(values,labels,validation_freq=0.3,batch_size=32,epochs=10)

Epoch 1/10
164/164 [==============================] - 2s 9ms/sample - loss: 0.1653 - accuracy: 0.9268
Epoch 2/10
164/164 [==============================] - 1s 8ms/sample - loss: 0.1365 - accuracy: 0.9512
Epoch 3/10
164/164 [==============================] - 1s 7ms/sample - loss: 0.1228 - accuracy: 0.9573
Epoch 4/10
164/164 [==============================] - 1s 7ms/sample - loss: 0.0948 - accuracy: 0.9573
Epoch 5/10
164/164 [==============================] - 1s 7ms/sample - loss: 0.0858 - accuracy: 0.9573
Epoch 6/10
164/164 [==============================] - 1s 7ms/sample - loss: 0.0853 - accuracy: 0.9573
Epoch 7/10
164/164 [==============================] - 1s 7ms/sample - loss: 0.0821 - accuracy: 0.9573
Epoch 8/10
164/164 [==============================] - 1s 7ms/sample - loss: 0.0829 - accuracy: 0.9512
Epoch 9/10
164/164 [==============================] - 1s 7ms/sample - loss: 0.0820 - accuracy: 0.9512
Epoch 10/10
164/164 [==============================] - 1s 7ms/sample - loss: 0.080